In [ ]:
!pip install -U torch torchvision --index-url https://download.pytorch.org/whl/cu124 && \
pip install -U langgraph langchain langchain-community sentence-transformers faiss-gpu-cu12 transformers accelerate bitsandbytes


In [ ]:

DB_PATH = '/content/movies_final.db'
COSINE_INDEX_FILE = '/content/movie_summaries_cosine.index'
CHUNK_MAP_FILE = '/content/chunk_map.pkl'

import sqlite3
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import numpy as np

try:
    embedding_model = SentenceTransformer('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
    faiss_index = faiss.read_index(COSINE_INDEX_FILE)
    with open(CHUNK_MAP_FILE, 'rb') as f:
        chunk_metadata = pickle.load(f)
    print("✅ Model ve indeks başarıyla yüklendi.")
except Exception as e:
    print(f"❌ HATA: Model veya indeks dosyaları yüklenemedi: {e}")
    exit()

In [ ]:
def sql_search_tool(query: str):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.close()
        if not results:
            return "SQL sorgusu sonuç döndürmedi."

        return results
    except Exception as e:
        return f"SQL Hatası: {e}"

def vector_search_tool(query: str, k: int = 5, min_similarity=0.4) -> list:
    try:
        index = faiss.read_index(COSINE_INDEX_FILE)
        query_embedding = embedding_model.encode(query, convert_to_tensor=False)
        query_embedding = np.array([query_embedding], dtype='float32')
        faiss.normalize_L2(query_embedding) 

        distances, indices = index.search(query_embedding, k * 3)

        found_titles = set()
        results = []
        for sim, idx in zip(distances[0], indices[0]):
            if sim < min_similarity or idx < 0:
                continue

            chunk_info = chunk_metadata[idx]
            movie_title = chunk_info['title']
            if movie_title not in found_titles:
                found_titles.add(movie_title)
                results.append(movie_title)

            if len(results) >= k:
                break

        if not results:
            return "Vektör araması anlamlı bir sonuç bulamadı."

        return results

    except FileNotFoundError:
        return f"Vektör Arama Hatası: İndeks dosyası ({COSINE_INDEX_FILE}) bulunamadı."
    except Exception as e:
        return f"Vektör Arama Hatası: {e}"

# def format_tool_results_for_llm(results: List[dict]) -> str:
#     if not results:
#         return "Daha önce bir araç çalıştırılmadı."

#     formatted_string = "Önceki Adımların Sonuçları:\n"
#     for res in results:
#         if "vector_result" in res:
#             # Vektör arama sonucunu formatla
#             movies = res['vector_result']
#             if isinstance(movies, list) and movies:
#                 formatted_string += f"- Anlamsal Arama, şu filmleri buldu: {', '.join(movies)}\n"
#             else:
#                 formatted_string += f"- Anlamsal Arama bir sonuç bulamadı.\n"

#         if "sql_result" in res:
#             # SQL sonucunu formatla
#             sql_output = res['sql_result']
#             if isinstance(sql_output, list) and sql_output:
#                 # [(Film1,), (Film2,)] formatını "Film1, Film2" haline getir
#                 cleaned_output = [item[0] for item in sql_output if isinstance(item, tuple) and item]
#                 formatted_string += f"- Veritabanı Sorgusu, şu sonuçları döndürdü: {', '.join(cleaned_output)}\n"
#             else:
#                  formatted_string += f"- Veritabanı Sorgusu bir sonuç bulamadı.\n"

#         if "router_decision" in res:
#             # Bu adımı atlayabilir veya loglama için tutabiliriz, LLM'in kafasını karıştırmasın.
#             pass # Karar adımlarını modele göstermeye gerek yok.

#     return formatted_string.strip()

def get_db_schema(db_path: str) -> str:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
    tables = cursor.fetchall()

    all_table_schemas = []
    for table_name_tuple in tables:
        table_name = table_name_tuple[0]

        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        column_names = [column[1] for column in columns]

        table_schema_str = f"{table_name}({','.join(column_names)})"
        all_table_schemas.append(table_schema_str)

    conn.close()
    return "; ".join(all_table_schemas)

db_schema = get_db_schema(DB_PATH)
print(db_schema)
print("\n--- Araç Testleri ---")
print("SQL Test:", sql_search_tool("SELECT title FROM movies WHERE release_date > '2000-01-01' LIMIT 2"))

movies(imdb_id,title,original_title,overview,tagline,release_date,runtime,original_language,adult); genres(id,name); production_companies(id,name); countries(id,name); spoken_languages(id,name); actors(id,name); crew_members(id,name); movie_genres(movie_imdb_id,genre_id); movie_companies(movie_imdb_id,company_id); movie_countries(movie_imdb_id,country_id); movie_languages(movie_imdb_id,language_id); movie_cast(movie_imdb_id,actor_id,character); movie_crew(movie_imdb_id,crew_member_id,job)

--- Araç Testleri ---
SQL Test: [('Spirited Away',), ('The Dark Knight',)]


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, AutoConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
print("Model ve tokenizer yükleniyor...")

model_id = "mmmmmmabel/cokelek-v3"
base_model_id = "unsloth/mistral-7b-v0.3"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
config = AutoConfig.from_pretrained(base_model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    config=config,
    quantization_config=quantization_config,
    device_map="auto" 
)


In [ ]:

text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.3,
    top_k=10,
    top_p=0.95,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("✅ Model başarıyla yüklendi ve LangChain pipeline'ı oluşturuldu!")

In [ ]:
from typing import TypedDict, List
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
import json
import re

class AgentState(TypedDict):
    original_query: str
    tool_results: List[dict]
    sql_attempts: List[str]
    generated_sql: str
    final_answer: str
    attempt_count: int

def parse_llm_decision(llm_output: str) -> str:
    llm_output = llm_output.lower()
    if "generate_sql" in llm_output:
        return "generate_sql"
    if "vector_search" in llm_output:
        return "vector_search"
    if "respond" in llm_output:
        return "respond"
    return "respond"

router_prompt_template = """Senin görevin bir sonraki adımı seçmektir. Sadece şu üç kelimeden birini kullanarak cevap ver: 'generate_sql', 'vector_search', 'respond'. Açıklama yapma. Sadece tek kelime.

-- KESİN KURALLAR --
1. Eğer 'Önceki Adımlar' içinde soruyu cevaplayan bir 'sql_result' veya 'vector_result' varsa, kararın KESİNLİKLE 'respond' olmalıdır.
2. Sorgu, belirli bir oyuncu adı, yönetmen adı, yıl gibi net veriler içeriyorsa, kararın 'generate_sql' olmalıdır.
3. Sorgu, bir sahneyi, temayı, atmosferi, karakteri veya film konusunu tarif ediyorsa, kararın 'vector_search' olmalıdır.
4. Önceki adımlar içerisinde cevap verebilmene yetecek bilgi var ise kararın 'respond' olmalıdır.

-- ÖRNEKLER --
# Örnek 1: Net veri içeren sorgu
Kullanıcının Sorusu: "Tom Hanks'in oynadığı filmler hangileri?"
Önceki Adımlar: []
Kararın: generate_sql

# Örnek 2: Sahne tarifi veya betimleme içeren anlam tabanlı sorgu
Kullanıcının Sorusu: "rüzgarların olduğu bir film vardı neydi adı?"
Önceki Adımlar: []
Kararın: vector_search

Kullanıcının Sorusu: "Yönetmeni Nuri Bilge Ceylan olan filmler hangileridir?"
Önceki Adımlar: [{{'router_decision': 'generate_sql'}}, {{'sql_result': "[('Kış Uykusu',)]"}}]
Kararın: respond

Kullanıcının Sorusu: "iki çocuğun olduğu film neydi?"
Önceki Adımlar: [{{'router_decision': 'vector_search'}}, {{'vector_result': "Bulunan film IMDb ID'leri: ['12 Angry Men', 'Pulp Fiction', 'The Godfather Part II', 'City of God', 'Impossible Things']"}}]
Kararın: respond
-- BİTTİ --

-- GÜNCEL DURUM --
Önceki Adımlar: {tool_results}
Kullanıcının Sorusu: {original_query}
Kararın:"""
router_prompt = PromptTemplate.from_template(router_prompt_template)

def router_node(state: AgentState):
    print("--- 1. Yönlendirici Düğüm ---")
    state['attempt_count'] += 1
    if state['attempt_count'] > 5:
        print("Maksimum deneme sayısına ulaşıldı. Cevap veriliyor.")
        return {"tool_results": state['tool_results'] + [{"router_decision": "respond"}]}
    prompt = router_prompt.format(original_query=state['original_query'], tool_results=state['tool_results'])
    llm_response = llm.invoke(prompt)
    clean_response = llm_response.strip().split('\n')[0].strip()
    decision = parse_llm_decision(clean_response)
    print(f"LLM Ham Çıktısı:\n{llm_response}\nTemizlenmiş Karar: {decision}")
    return {"tool_results": state['tool_results'] + [{"router_decision": decision}]}

sql_generator_prompt_template = """
Görevin, verilen veritabanı şemasına ve kullanıcı sorusuna dayanarak %100 doğru ve geçerli bir SQLite sorgusu oluşturmaktır.

VERİTABANI ŞEMASI:
{db_schema}

Sorgu oluştururken uyman gereken KESİN KURALLAR:
1.  Bir kişiyi (yönetmen veya oyuncu) bir filme bağlamak için ASLA `movies` tablosunu doğrudan `actors` veya `crew_members` tablosuna bağlama. Bu imkansızdır.
2.  Bağlantı için her zaman ara tabloları (`movie_cast` veya `movie_crew`) kullanmalısın.
3.  **Yönetmen bulmak için İZLENECEK YOL:** `movies` tablosunu `movie_crew` tablosuna, sonra `movie_crew` tablosunu `crew_members` tablosuna bağla.
    - `movies.imdb_id` = `movie_crew.movie_imdb_id`
    - `movie_crew.crew_member_id` = `crew_members.id`
    - Ayrıca, `movie_crew.job = 'Director'` koşulunu KESİNLİKLE eklemelisin.
4.  **Oyuncu bulmak için İZLENECEK YOL:** `movies` tablosunu `movie_cast` tablosuna, sonra `movie_cast` tablosunu `actors` tablosuna bağla.
    - `movies.imdb_id` = `movie_cast.movie_imdb_id`
    - `movie_cast.actor_id` = `actors.id`

BAŞARISIZ DENEMELER (Sorgu ve Hata):
{sql_attempts}

Kullanıcının Sorusu: {original_query}

Yukarıdaki katı kurallara ve önceki hatalardan çıkardığın derslere dayanarak, doğru SQL sorgusunu oluştur. Sadece SQL kodunu yaz.
Doğru SQL Sorgusu:
"""
sql_generator_prompt = PromptTemplate.from_template(sql_generator_prompt_template)

def sql_generator_node(state: AgentState):
    print("--- 2a. Kural Tabanlı ve Akıllı Temizleyici SQL Üretici Düğüm ---")
    db_schema = get_db_schema(DB_PATH)
    attempts_str = "\n".join(state.get('sql_attempts', []))
    prompt = sql_generator_prompt.format(
        db_schema=db_schema,
        original_query=state['original_query'],
        sql_attempts=attempts_str
    )
    generated_text = llm.invoke(prompt).strip()

    code_block_match = re.search(r"```(?:\w*\n)?(.*?)```", generated_text, re.DOTALL)

    if code_block_match:
        generated_sql = code_block_match.group(1).strip()
    else:
        generated_sql = generated_text

    print(f"LLM Ham Çıktısı:\n{generated_text}")
    print(f"Temizlenmiş SQL: {generated_sql}")

    return {"generated_sql": generated_sql}


def sql_executor_node(state: AgentState):
    print("--- 2b. SQL Çalıştırıcı Düğüm ---")
    sql_query = state['generated_sql']
    result = sql_search_tool(sql_query)
    if "SQL Hatası" in result:
        failed_attempt = f"Sorgu: {sql_query}\nHata: {result}"
        return {"sql_attempts": state.get('sql_attempts', []) + [failed_attempt]}
    else:
        if isinstance(result, str) and "sonuç döndürmedi" in result:
             failed_attempt = f"Sorgu: {sql_query}\nHata: Sorgu çalıştı ama hiç sonuç bulamadı. Bu, JOIN mantığında bir hata olduğunu gösterebilir."
             return {"sql_attempts": state.get('sql_attempts', []) + [failed_attempt], "tool_results": state['tool_results'] + [{"sql_result": result}]}
        return {"tool_results": state['tool_results'] + [{"sql_result": result}], "sql_attempts": []}

def vector_search_node(state: AgentState):
    print("--- 3. Vektör Arama Düğümü ---")
    result = vector_search_tool(state['original_query'])
    return {"tool_results": state['tool_results'] + [{"vector_result": result}]}

response_synthesizer_prompt_template = """Sen son kullanıcıya doğrudan cevap veren bir film asistanısın. Görevin, aşağıdaki teknik verileri analiz ederek kullanıcıya SADECE akıcı ve doğal bir dilde cevap vermektir. Kesinlikle kod, ```python``` bloğu veya teknik bir açıklama üretme. Sadece son kullanıcının göreceği cevabı yaz.
-- ÖRNEK --
Teknik Veriler: [{{'sql_result': "[('Forrest Gump',), ('Cast Away',)]"}}]
Kullanıcı Sorusu: Tom Hanks filmleri?
İstenen Cevap: Elbette, Tom Hanks'in oynadığı bazı filmler şunlardır: Forrest Gump ve Cast Away.
-- BİTTİ --
-- GÜNCEL DURUM --
Toplanan Bilgiler: {tool_results}
Kullanıcının Orijinal Sorusu: {original_query}
İstenen Cevap:"""
response_synthesizer_prompt = PromptTemplate.from_template(response_synthesizer_prompt_template)

def response_synthesizer_node(state: AgentState):
    print("--- 4. Yanıt Sentezleyici Düğüm ---")
    successful_results = [r for r in state['tool_results'] if (isinstance(r.get('sql_result'), list) and r.get('sql_result')) or (isinstance(r.get('vector_result'), list) and r.get('vector_result'))]
    if not successful_results:
        final_answer = "Üzgünüm, birkaç denemeye rağmen sorunuza uygun bir cevap bulamadım. Lütfen sorunuzu farklı bir şekilde sormayı deneyin."
    else:
        prompt = response_synthesizer_prompt.format(tool_results=state['tool_results'], original_query=state['original_query'])
        final_answer = llm.invoke(prompt)
    return {"final_answer": final_answer}

def route_logic(state: AgentState):
    last_decision = state['tool_results'][-1].get("router_decision")
    if "generate_sql" in last_decision:
        return "generate_sql"
    elif "vector_search" in last_decision:
        return "vector_search"
    else:
        return "respond"

def after_sql_execution(state: AgentState):
    if state.get('sql_attempts') and len(state.get('sql_attempts')) > 0:
        return "router"
    else:
        return "router"

graph = StateGraph(AgentState)

graph.add_node("router", router_node)
graph.add_node("sql_generator", sql_generator_node)
graph.add_node("sql_executor", sql_executor_node)
graph.add_node("vector_searcher", vector_search_node)
graph.add_node("responder", response_synthesizer_node)

graph.set_entry_point("router")

graph.add_conditional_edges(
    "router",
    route_logic,
    {
        "generate_sql": "sql_generator",
        "vector_search": "vector_searcher",
        "respond": "responder",
    }
)
graph.add_edge("sql_generator", "sql_executor")
graph.add_conditional_edges(
    "sql_executor",
    after_sql_execution,
    {
        "router": "router",
    }
)
graph.add_edge("vector_searcher", "router")
graph.add_edge("responder", END)

app = graph.compile()
print("\n✅ LangGraph uygulaması SON ve TEMİZ MİMARİ ile başarıyla derlendi!")


✅ LangGraph uygulaması SON ve TEMİZ MİMARİ ile başarıyla derlendi!


In [ ]:
import sqlite3
import nest_asyncio
from fastapi import FastAPI, Request
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware

class QueryRequest(BaseModel):
    query: str

def run_agent(query):
    initial_state = {
        "original_query": query,
        "tool_results": [],
        "sql_attempts": [],
        "generated_sql": "",
        "final_answer": "",
        "attempt_count": 0
    }
    for s in app.stream(initial_state, {"recursion_limit": 15}):
        print("---")
        print(list(s.values())[0])

    final_state = list(s.values())[0]
    return final_state.get("final_answer", "Bir sonuca ulaşılamadı.")

    final_result = run_agent(query)
api_app = FastAPI()

api_app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=False, 
    allow_methods=["GET", "POST", "OPTIONS"],
    allow_headers=["*"], 
    max_age=86400, 
)

@api_app.options("/agent-query/", include_in_schema=False)
async def options_agent_query():
    return {} 

@api_app.post("/agent-query/")
async def process_query(request: QueryRequest):
    print(f"API'den gelen sorgu: '{request.query}'")
    final_answer = run_agent(request.query)
    print(f"Ajanın ürettiği son cevap: '{final_answer}'")
    return {"response": final_answer}

@api_app.middleware("http")
async def log_requests(request: Request, call_next):
    print(f"İstek alındı: {request.method} {request.url}")
    print(f"İstek başlıkları: {request.headers}")
    response = await call_next(request)
    print(f"Yanıt durumu: {response.status_code}")
    return response

nest_asyncio.apply()
try:
    ngrok_authtoken = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(ngrok_authtoken)
    public_url = ngrok.connect(8000)
    print(f"✅ Ngrok Tüneli Aktif: {public_url}")
    uvicorn.run(api_app, host="0.0.0.0", port=8000)
except Exception as e:
    print(f"❌ Hata: {e}")